<a href="https://colab.research.google.com/github/deepakri201/SR_for_line/blob/main/SR_for_line.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a demo notebook to create an SR with a line (polyline) instead of a bounding box.

Deepa Krishnaswamy

Brigham and Women's Hospital

April 2025

# Install highdicom from github

In [ ]:
!git clone https://github.com/imagingdatacommons/highdicom ~/highdicom

Cloning into '/root/highdicom'...
remote: Enumerating objects: 9845, done.
remote: Counting objects: 100% (1326/1326), done.
remote: Compressing objects: 100% (318/318), done.
remote: Total 9845 (delta 1115), reused 1057 (delta 989), pack-reused 8519 (from 3)
Receiving objects: 100% (9845/9845), 8.16 MiB | 3.75 MiB/s, done.
Resolving deltas: 100% (6516/6516), done.


In [ ]:
!pip install ~/highdicom

Processing /root/highdicom
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 70.1 MB/s eta 0:00:00
  Created wheel for highdicom: filename=highdicom-0.25.1-py3-none-any.whl size=1090733 sha256=285fed2f5c97eee2623c3c9db0c85f96a40c34a5ffdfd526da7d65d8dbc1d3c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-yu8z9n62/wheels/81/f9/ce/c8968ff70c7966d79e971c7c2c166595b6f28db27bb129d477
Successfully built highdicom


# Parameterization

In [ ]:
#@title Enter your Project ID here
# initialize this variable with your Google Cloud Project ID!
project_name = "idc-external-018" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = project_name

!gcloud config set project $project_name

from google.colab import auth
auth.authenticate_user()

Updated property [core/project].


In [ ]:
location = 'us-central1'
# bucket_sr_directory = "gs://nlst_cancer_annot/sr"
bucket_sr_directory = "gs://nlst_cancer_annot/sr_multiple_bbox_per_sop"
# bucket_sr_tracked_directory = "gs://nlst_cancer_annot/sr_tracked"

# Environment setup

In [ ]:
import os

# Authorize
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 13.8 MB/s eta 0:00:00


In [ ]:
!pip install pydicom

In [ ]:
!pip install idc-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.2.2
    Uninstalling duckdb-1.2.2:
      Successfully uninstalled duckdb-1.2.2


In [ ]:
import os
import sys
import time

from google.cloud import bigquery
from google.cloud import storage

import numpy as np
import pandas as pd
import nibabel as nib
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

import json
from pathlib import Path

import pydicom
from pydicom.uid import generate_uid
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes

In [ ]:
#!pip install highdicom

In [ ]:
from highdicom.sr.content import (
    FindingSite,
    ImageRegion,
    ImageRegion3D,
    SourceImageForRegion,
    SourceImageForMeasurement
)

from highdicom.sr.enum import GraphicTypeValues3D
from highdicom.sr.enum import GraphicTypeValues
from highdicom.sr.sop import Comprehensive3DSR, ComprehensiveSR
from highdicom.sr.templates import (
    DeviceObserverIdentifyingAttributes,
    Measurement,
    MeasurementProperties,
    MeasurementReport,
    ObservationContext,
    ObserverContext,
    PersonObserverIdentifyingAttributes,
    PlanarROIMeasurementsAndQualitativeEvaluations,
    RelationshipTypeValues,
    TrackingIdentifier,
    QualitativeEvaluation
)
# from highdicom.sr.value_types import (
#     CodedConcept,
#     CodeContentItem,
# )
# why from highdicom.sr.value_types for CodedConcept? Should be highdicom.sr.CodedConcept
from highdicom.sr import CodedConcept

from highdicom.sr import ImageLibrary
from highdicom.sr import ImageLibraryEntryDescriptors
from highdicom.sr.value_types import Code
from highdicom.sr.value_types import ContentItem
from highdicom.sr import PixelOriginInterpretationValues

import logging
logger = logging.getLogger("highdicom.sr.sop")
logger.setLevel(logging.INFO)

In [ ]:
import highdicom as hr
hr.__version__

'0.25.1'

In [ ]:
pydicom.__version__

'3.0.1'

In [ ]:
from idc_index import IDCClient

client = IDCClient.client()

# Create a SR with a polyline

In [ ]:
def save_SR_for_case_polyline(sop_files,
                              markups_sop_df,
                              path_sr):
  """
  Summary:
  This function creates a saves a SR with polylines. (not a bounding box)

  Inputs:
    sop_files       - all of the image filenames
    markups_sop_df  - a dataframe holding all the info for the lines
    path_sr         - filename for the output SR

  Outputs:
  path_sr is saved to disk.

  """

  # Read CT Image data set from PS3.10 files on disk
  # image_dataset = dcmread(sop_filename_keep)

  # image_dataset = dcmread(sop_filenames_keep[0]) # can be any?
  # provide all frames for image_dataset instead of just one
  image_dataset = [pydicom.dcmread(f, stop_before_pixels=True) for f in sop_files]

  # Get the frame_of_reference_uid
  dcm = pydicom.dcmread(sop_files[0])
  frame_of_reference_uid = dcm.FrameOfReferenceUID
  referenced_sop_class_uid = dcm.SOPClassUID

  # Describe the context of reported observations: the person that reported
  # the observations and the device that was used to make the observations
  observer_person_context = ObserverContext(
      observer_type=codes.DCM.Person,
      observer_identifying_attributes=PersonObserverIdentifyingAttributes(
          name='Anonymous^Reader'
      )
  )
  observer_device_context = ObserverContext(
      observer_type=codes.DCM.Device,
      observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
          uid=generate_uid()
      )
  )
  observation_context = ObservationContext(
      observer_person_context=observer_person_context,
      #observer_device_context=observer_device_context,
  )

  imaging_measurements = []

  # Get the number of TrackingIdentifiers in the markups_sop_dataframe
  # This determines the number of lines
  tracking_identifiers = sorted(list(set(markups_sop_df['TrackingIdentifier'].values)))
  num_of_tracking_identifiers = len(tracking_identifiers)
  print('num_of_tracking_identifiers: ' + str(num_of_tracking_identifiers))

  #########################
  ### Add for each line ###
  #########################

  for tracking_identifier in tracking_identifiers:

    # Get the dataframe for this line
    markups_sop_df_subset = markups_sop_df[markups_sop_df['TrackingIdentifier']==tracking_identifier]

    # Get the SOPInstanceUID
    sop_instance_uid = markups_sop_df_subset['SOPInstanceUID'].values[0]

    # Get the finding_site
    finding_location_value = markups_sop_df_subset['FindingSite'].values[0]
    print('finding_location_value: ' + str(finding_location_value))

    # Get the finding_type
    finding_type_value = markups_sop_df_subset['LengthMeasurement'].values[0]

    # Get the point
    graphic_data = np.asarray ( [ [np.float32(markups_sop_df_subset['pointx'].values[0]), np.float32(markups_sop_df_subset['pointy'].values[0])],
                                  [np.float32(markups_sop_df_subset['pointx'].values[1]), np.float32(markups_sop_df_subset['pointy'].values[1])] ])

    # Set the finding_sites
    finding_location = CodedConcept(
                                    value=finding_location_value[0],
                                    scheme_designator=finding_location_value[1],
                                    meaning=finding_location_value[2]
    )
    finding_sites = [
        FindingSite(anatomic_location=finding_location)
    ]

    # Set the finding_type
    finding_type = CodedConcept(
                                value=finding_type_value[0],
                                scheme_designator=finding_type_value[1],
                                meaning=finding_type_value[2]
    )

    # (121055, DCM, "Path")
    # Create the linear measurement
    image_region = ImageRegion(
                            graphic_data=graphic_data,
                            graphic_type=GraphicTypeValues.POLYLINE,
                            source_image=SourceImageForRegion(referenced_sop_class_uid=referenced_sop_class_uid,
                                                              referenced_sop_instance_uid=sop_instance_uid),
                            pixel_origin_interpretation=PixelOriginInterpretationValues.VOLUME
    )
    # print('image_region: ' + str(image_region))

    TrackingUID = generate_uid()

    imaging_measurements.append(
        PlanarROIMeasurementsAndQualitativeEvaluations(
            tracking_identifier=TrackingIdentifier(
                uid=TrackingUID,
                identifier=tracking_identifier
            ),
            referenced_region=image_region,
            finding_type=finding_type,
            finding_sites=finding_sites,
        )
    )


  # print('imaging_measurements: ' + str(imaging_measurements))

  # Create the report content
  procedure_code = CodedConcept(value="363679005",
                                scheme_designator="SCT",
                                meaning="Imaging procedure")
  measurement_report = MeasurementReport(
      observation_context=observation_context,
      procedure_reported=procedure_code,
      imaging_measurements=imaging_measurements
  )

  # Create the Structured Report instance
  sr_dataset = Comprehensive3DSR(
      evidence=image_dataset,
      content=measurement_report,
      series_number=100,
      series_instance_uid=generate_uid(),
      sop_instance_uid=generate_uid(),
      series_description = "LV and Aorta lines",
      instance_number=1,
      manufacturer='Deepa',
      is_complete = True,
      is_final=True
  )

  sr_dataset.save_as(path_sr, enforce_file_format=True)

  return sr_dataset

In [ ]:
# Download test data
SeriesInstanceUID = "1.2.840.113654.2.55.97114726565566537928831413367474015470"
# The SEG that corresponds with this file is:
# SeriesInstanceUID = "1.2.276.0.7230010.3.1.3.313263360.40027.1706310312.485948"

# I created the markups in Slicer myself
# Read from the google drive

markups_directory = "/content/gdrive/MyDrive/Colab Notebooks/Sybil_NLST/markups"
lv_1 = ["LV1A", "LV1B"]
lv_2 = ["LV2A", "LV2B"]
lv_3 = ["LV3A", "LV3B"]
aorta_1 = ["Aorta1A", "Aorta1B"]
aorta_2 = ["Aorta2A", "Aorta2B"]
aorta_3 = ["Aorta3A", "Aorta3B"]

# https://dicom.nema.org/medical/dicom/2022a/output/chtml/part16/sect_CID_7470.html
lv_measurement = ["103339001", "SCT", "Long axis"]
aorta_measurement = ["81827009", "SCT", "Diameter"]

# get the filenames
markups_filenames = [os.path.join(markups_directory,f) for f in os.listdir(markups_directory)]
# read the files - coordinates are already in LPS
markups_dict = {}
for n in range(0,len(markups_filenames)):
  with open(markups_filenames[n]) as f:
    data = json.load(f)
    name = data['markups'][0]['controlPoints'][0]['label']
    point = data['markups'][0]['controlPoints'][0]['position']
    markups_dict.update({name: point})

# Sort the keys
markups_dict = {key: markups_dict[key] for key in sorted(markups_dict.keys())}

In [ ]:
# now create a dataframe with the following info:
# Tracking ID, FindingSite, point

markups_df = pd.DataFrame()
# markups_df['TrackingIdentifier'] = [1,1,2,2,3,3,4,4,5,5,6,6]
markups_df['TrackingIdentifier'] = ['Aorta #1', 'Aorta #1',
                                    'Aorta #2', 'Aorta #2',
                                    'Aorta #3', 'Aorta #3',
                                    'LV #1', 'LV #1',
                                    'LV #2', 'LV #2',
                                    'LV #3', 'LV #3']
markups_df['FindingSite'] = 6 * [["15825003", "SCT", "Aortic structure", ]] + 6 * [["87878005", "SCT", "Left cardiac ventricular structure", ]]
markups_df['LengthMeasurement'] = 6 * [lv_measurement] + 6 * [aorta_measurement]
markups_df['pointx'] = [np.float32(f[0]) for f in markups_dict.values()]
markups_df['pointy'] = [np.float32(f[1]) for f in markups_dict.values()]
markups_df['pointz'] = [np.float32(f[2]) for f in markups_dict.values()]

In [ ]:
# Create dicom directory
dicom_directory = "/content/ct"
if not os.path.isdir(dicom_directory):
  os.mkdir(dicom_directory)

# Download series
client.download_from_selection(
    seriesInstanceUID=SeriesInstanceUID,
    downloadDir=dicom_directory,
    dirTemplate='',
    quiet=True
)

# Get the list of files
sop_files = os.listdir("/content/ct")
sop_files = [os.path.join("/content/ct",f) for f in sop_files]
print('Number of CT files: ' + str(len(sop_files)))

Number of CT files: 129


In [ ]:
# Now get all of the SOPInstanceUIDs for the entire CT dataset
# And get the IPP for each slice
SOPInstanceUID_list_all = []
IPP_list_all = []
for index,sop_file in enumerate(sop_files):
  sop_data = pydicom.dcmread(sop_file)
  SOPInstanceUID_list_all.append(sop_data.SOPInstanceUID)
  IPP_list_all.append(sop_data.ImagePositionPatient)
  if (index==0):
    FrameOfReferenceUID = sop_data.FrameOfReferenceUID

print('SOPInstanceUID_list_all: ' + str(SOPInstanceUID_list_all))
print('IPP_list_all: ' + str(IPP_list_all))
print('FrameOfReferenceUID: ' + str(FrameOfReferenceUID))
print(len(SOPInstanceUID_list_all))

SOPInstanceUID_list_all: ['1.2.840.113654.2.55.196105592429989324462564512858407189915', '1.2.840.113654.2.55.129436140110320975521311108287202866154', '1.2.840.113654.2.55.6430606331522375512402226918214298643', '1.2.840.113654.2.55.239138574486451560396217541383989878670', '1.2.840.113654.2.55.165433639898562980173950168277731204041', '1.2.840.113654.2.55.107757470855854749188445422292881179919', '1.2.840.113654.2.55.319287716406378428288949135956301309603', '1.2.840.113654.2.55.275993473711046844651391827491575843890', '1.2.840.113654.2.55.100413131866180443912885120934685487196', '1.2.840.113654.2.55.198943385182066297090474108471491240832', '1.2.840.113654.2.55.101765809332232202300479570726046762632', '1.2.840.113654.2.55.14540425696228821121109700771431170258', '1.2.840.113654.2.55.250753576478297700484480795310710343017', '1.2.840.113654.2.55.207040379262405968637007896980071558451', '1.2.840.113654.2.55.249620618757778529123433324458549215980', '1.2.840.113654.2.55.69936263189

In [ ]:
# Now add to the dataframe the SOPInstanceUID, and the filename, by matching the IPP
IPP0_list = [np.float32(IPP[0]) for IPP in IPP_list_all]
IPP1_list = [np.float32(IPP[1]) for IPP in IPP_list_all]
IPP2_list = [np.float32(IPP[2]) for IPP in IPP_list_all]

sop_df = pd.DataFrame()
sop_df['SOPInstanceUID'] = SOPInstanceUID_list_all
sop_df['filename'] = sop_files
sop_df['ipp0'] = IPP0_list
sop_df['ipp1'] = IPP1_list
sop_df['ipp2'] = IPP2_list

In [ ]:
# Now join the markups_df and the sop_df on the ipp2
markups_sop_df = pd.merge(markups_df, sop_df, left_on='pointz', right_on='ipp2', how='inner')
markups_sop_df

,TrackingIdentifier,FindingSite,LengthMeasurement,pointx,pointy,pointz,SOPInstanceUID,filename,ipp0,ipp1,ipp2
0,Aorta #1,"[15825003, SCT, Aortic structure]","[103339001, SCT, Long axis]",13.933864,62.622200,-277.225006,1.2.840.113654.2.55.25075357647829770048448079...,/content/ct/43e00584-f007-4f72-ac3d-b453e95fe1...,-170.699997,-180.0,-277.225006
1,Aorta #1,"[15825003, SCT, Aortic structure]","[103339001, SCT, Long axis]",38.598587,63.146980,-277.225006,1.2.840.113654.2.55.25075357647829770048448079...,/content/ct/43e00584-f007-4f72-ac3d-b453e95fe1...,-170.699997,-180.0,-277.225006
2,Aorta #2,"[15825003, SCT, Aortic structure]","[103339001, SCT, Long axis]",17.082552,67.345230,-249.725006,1.2.840.113654.2.55.10015000131048938759118361...,/content/ct/a88d1d31-b8ab-425f-9a9f-00ee8b4bcb...,-170.699997,-180.0,-249.725006
3,Aorta #2,"[15825003, SCT, Aortic structure]","[103339001, SCT, Long axis]",39.123367,67.870010,-249.725006,1.2.840.113654.2.55.10015000131048938759118361...,/content/ct/a88d1d31-b8ab-425f-9a9f-00ee8b4bcb...,-170.699997,-180.0,-249.725006
4,Aorta #3,"[15825003, SCT, Aortic structure]","[103339001, SCT, Long axis]",18.656895,68.394791,-239.725006,1.2.840.113654.2.55.69936263189562817383481115...,/content/ct/8db12c22-abd7-4554-82b6-478812bc98...,-170.699997,-180.0,-239.725006
5,Aorta #3,"[15825003, SCT, Aortic structure]","[103339001, SCT, Long axis]",42.272057,68.394791,-239.725006,1.2.840.113654.2.55.69936263189562817383481115...,/content/ct/8db12c22-abd7-4554-82b6-478812bc98...,-170.699997,-180.0,-239.725006
6,LV #1,"[87878005, SCT, Left cardiac ventricular struc...","[81827009, SCT, Diameter]",43.321617,16.966221,-277.225006,1.2.840.113654.2.55.25075357647829770048448079...,/content/ct/43e00584-f007-4f72-ac3d-b453e95fe1...,-170.699997,-180.0,-277.225006
7,LV #1,"[87878005, SCT, Left cardiac ventricular struc...","[81827009, SCT, Diameter]",78.481972,-26.065847,-277.225006,1.2.840.113654.2.55.25075357647829770048448079...,/content/ct/43e00584-f007-4f72-ac3d-b453e95fe1...,-170.699997,-180.0,-277.225006
8,LV #2,"[87878005, SCT, Left cardiac ventricular struc...","[81827009, SCT, Diameter]",33.875553,23.263597,-249.725006,1.2.840.113654.2.55.10015000131048938759118361...,/content/ct/a88d1d31-b8ab-425f-9a9f-00ee8b4bcb...,-170.699997,-180.0,-249.725006
9,LV #2,"[87878005, SCT, Left cardiac ventricular struc...","[81827009, SCT, Diameter]",71.659813,-12.421534,-249.725006,1.2.840.113654.2.55.10015000131048938759118361...,/content/ct/a88d1d31-b8ab-425f-9a9f-00ee8b4bcb...,-170.699997,-180.0,-249.725006


In [ ]:
# Now create the SR

path_sr = "/content/sr_polyline.dcm"

sr = save_SR_for_case_polyline(sop_files,
                               markups_sop_df,
                               path_sr)

print(sr)

num_of_tracking_identifiers: 6
finding_location_value: ['15825003', 'SCT', 'Aortic structure']
finding_location_value: ['15825003', 'SCT', 'Aortic structure']
finding_location_value: ['15825003', 'SCT', 'Aortic structure']
finding_location_value: ['87878005', 'SCT', 'Left cardiac ventricular structure']
finding_location_value: ['87878005', 'SCT', 'Left cardiac ventricular structure']
finding_location_value: ['87878005', 'SCT', 'Left cardiac ventricular structure']
Dataset.file_meta -------------------------------
(0002,0000) File Meta Information Group Length  UL: 226
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0002) Media Storage SOP Class UID         UI: Comprehensive 3D SR Storage
(0002,0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.8.498.88026608177869181263991123815001036925
(0002,0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002,0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.9.7433.1.1
(00

In [ ]:
# Now download so I can validate

from google.colab import files

!zip -r /content/ct.zip /content/ct
files.download("/content/ct.zip")
files.download("/content/sr_polyline.dcm")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>